# Using Machine Learning to Help Small Businesses
If there is one thing businesses are learning, it's that proper data analytics isn't enough to gain a competetive edge, but has become a necessary ticket to compete. Sadly, many small businesses have neither the resources nor the know-how to take advantage of what data is available to them. With this project, the aim is to build a machine learning algorithm that can help any small business get a better sense of how they are presenting themselves to customers, who their direct competitors are, and how they compare to those competitors.

## Yelp
This primary source of data for this analysis is Yelp. We will use the Yelp API in conjunction with web scraping techniques and NLP sentiment analysis to gather and analyze customer feedback.

In [1]:
import pandas as pd
import numpy as np
import json
from yelpapi import YelpAPI
from sklearn.cluster import KMeans

# Load Yelp API token data
f = open('access_tokens.json', 'r')
tokens = json.load(f)
section_tokens = tokens['yelp']['first_yelp_app_rwilleyii']
client_id = section_tokens['Client ID']
api_key = section_tokens['API Key']

# Load Yelp API
yelp_api = YelpAPI(api_key)

## Business Selection Criteria
This model relies on 3 inputs from the user to generate results:

1. **Company Name**: This must be input exactly how the name appears on Yelp. It is advised to use copy/paste directly from Yelp.
2. **Search Term**: Consider a customer that doesn't know the business exists, but could benefit from the service/product the company offers. What search terms do you expect your generic customer to search for? (I.e "mexican food", "rooftop bar", "office supplies", etc.)
3. **Location**: Enter a location in the way you expect most customers to search. In most cases, this will be simply City, ST, but it could be zip cod as well.

In [2]:
# Get user preferences
company_name = input('Please enter company name as it appears on Yelp.com: ')
term = input('For someone who has never been to your restaurant, what terms might they search on Yelp to find you? ')
location = input('Where is your restaurant located? (city ST, ZIP, etc.) ')

Please enter company name as it appears on Yelp.com:  Chano's Cantina
For someone who has never been to your restaurant, what terms might they search on Yelp to find you?  mexican bar
Where is your restaurant located? (city ST, ZIP, etc.)  astoria, ny


In [3]:
company_details = pd.DataFrame(data=[[company_name, term, location]], 
                               columns=['company_name', 'search_terms', 'location'])
company_details.to_csv('firm_data/inputs.csv')

## Generate Results from Yelp API
Using the variables above, return the top 50 results. The results should include the business in question; if not, there are likely more fundamental issues that need to be addressed before this model will be helpful. For example, if you are an italian restaurant but don't show up in search results for italian restaurants in the same neighborhood, the company is probably improperly listed on Yelp. The company should check to make sure the proper tags are in place or contact Yelp to find out what is going on.

In [4]:
search_limit = 50

url = 'https://api.yelp.com/v3/businesses/search'

headers = {'Authorization': 'Bearer {}'.format(api_key)}

url_params = {'term': term.replace(' ', '+'),
              'location': location.replace(' ', '+'),
              'limit': search_limit
             }
response = yelp_api.search_query(term = term,
                                 location = location,
                                 limit = search_limit)

### Save Search Results to Dataframe

In [5]:
cols = list(response['businesses'][0].keys())
data = pd.DataFrame(columns=cols)

for biz in response['businesses']:
    data = data.append(biz, ignore_index=True)

data = data[data['is_closed']==False]
data.head(5)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,anBWvFPOce4nl3qLX-H6Rg,el-mero-mero-long-island-city,El Mero Mero,https://s3-media2.fl.yelpcdn.com/bphoto/7tp30k...,False,https://www.yelp.com/biz/el-mero-mero-long-isl...,81,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 40.753834323302, 'longitude': -73...","[pickup, delivery]",$$,"{'address1': '35-57 38th St', 'address2': '', ...",+17183922444,(718) 392-2444,1457.114073
1,6AJwsgXr7YwsqneGVAdgzw,las-catrinas-mexican-bar-and-eatery-astoria,Las Catrinas Mexican Bar & Eatery,https://s3-media3.fl.yelpcdn.com/bphoto/3uevye...,False,https://www.yelp.com/biz/las-catrinas-mexican-...,301,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 40.7614214682633, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '32-02 Broadway', 'address2': '',...",+19177450969,(917) 745-0969,642.525771
2,AUyKmFjpaVLwc3awfUnqgQ,chela-and-garnacha-astoria,Chela & Garnacha,https://s3-media1.fl.yelpcdn.com/bphoto/ChVbA1...,False,https://www.yelp.com/biz/chela-and-garnacha-as...,364,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.5,"{'latitude': 40.7557171543477, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '33-09 36th Ave', 'address2': '',...",+19178326876,(917) 832-6876,1318.326547
3,jeWIYbgBho9vBDhc5S1xvg,chanos-cantina-astoria,Chano's Cantina,https://s3-media3.fl.yelpcdn.com/bphoto/Di8o5L...,False,https://www.yelp.com/biz/chanos-cantina-astori...,164,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.0,"{'latitude': 40.756621, 'longitude': -73.929336}","[pickup, delivery, restaurant_reservation]",$$,"{'address1': '35-55 31st', 'address2': '', 'ad...",+19178327261,(917) 832-7261,1290.427488
4,O7CnII31QnhLIYoUlOXrZA,la-sala-astoria,La Sala,https://s3-media2.fl.yelpcdn.com/bphoto/DZKzPR...,False,https://www.yelp.com/biz/la-sala-astoria?adjus...,117,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.5,"{'latitude': 40.7632006, 'longitude': -73.9286...",[],$$,"{'address1': '3205 Crescent St', 'address2': '...",+17186066121,(718) 606-6121,678.809168


In [6]:
data.to_csv(f'firm_data/{company_name}_search_results.csv')